In [0]:
from pyspark.sql.functions import *

In [0]:
# reading silver and zone lookup table
df_silver=spark.table('nyc_taxi.nyc_schema.silver')
df_zone_lookup=spark.table('nyc_taxi.nyc_schema.zone_lookup')

In [0]:
df_gold_zone_hour=df_silver.join(broadcast(df_zone_lookup),df_silver.PULocationID==df_zone_lookup.location_id,how='left')

In [0]:
df_trip_metrics_by_zone_hour=df_gold_zone_hour.groupBy('pickup_date','pickup_hour','zone').agg(
    count('*').alias('trip_count'),
    round(sum('fare_amount'),2).alias('total_fare'),
    round(avg('tip_percent'),2).alias('avg_tip_percent')
).orderBy('pickup_date','pickup_hour','zone')
display(df_trip_metrics_by_zone_hour)

In [0]:
df_trip_metrics_by_zone_hour.write.mode('overwrite').saveAsTable('nyc_taxi.nyc_schema.trip_metrics_by_zone_hour')

In [0]:
%sql
select count(*) from nyc_taxi.nyc_schema.trip_metrics_by_zone_hour